In [1]:
import openai
import time
import json
import backoff
from openai.error import RateLimitError
import datetime
import pickle
import numpy as np
from numpy.linalg import norm
import re

In [2]:
openai.api_key=os.getenv("OPENAI_API_KEY")

In [3]:
s = open("/Users/nehasheth/Desktop/Research/Research - AI Chatbot TA/github/data-generator/QA_generation_from_textbook/gpt-3/GPT-3_section_level.json")
sections_data = json.load(s)

#full textbook embeddings - vectors
with open("/Users/nehasheth/Desktop/Research/Research - AI Chatbot TA/github/data-generator/QA_generation_from_textbook/reinforcement_learning/index.json") as input_file:
    index_data = json.load(input_file)



In [4]:
len(sections_data)

144

In [5]:
@backoff.on_exception(backoff.expo, RateLimitError)
def response_API(prompt, myKwargs = {}):

  #default arguments to send the API, unless changed in function
  kwargs = {"model" :"text-davinci-003",
            "temperature" :0.6,
            "max_tokens": 500,
            "frequency_penalty":1,
            "presence_penalty":0}


  for kwarg in myKwargs:
    kwargs[kwarg] = myKwargs[kwarg]

  r = openai.Completion.create(prompt=prompt, **kwargs)
  return r['choices'][0]['text']

In [59]:
def generate_answers(ques):
    prompts = [
        
        """Generate an objective and logically sound answer to this question. The answer should have a freshmen-level tone and be concise and very short.
        The answer must spur curiosity. Give examples.
        Question :
        """,

       """Generate a reasoning-based answer about this question. The answer should have a freshmen-level tone and be concise and very short.
        Give examples. It should be logic-oriented. Think step by step. Question : """ ,

        """Generate a list-type, short answer to this question. 
        It should be very short and concise and have a high school level language. Question : """  + "\nQuestion:",

        """Generate a detailed, long, interesting answer to this question. 
        The answer must be engaging and provoke interactions. It should use fresh men level language and a formal tone. Question : """
    ]

    n=len(prompts)
    answers = []
    for j in prompts:
        
        p= "Instruction : %s %s" % (j, ques)
    
        response = response_API(p)
        
        answers.append(response)
        print(response)

    
    return answers

In [7]:
#manually cleaned 
file1 = open("/Users/nehasheth/Desktop/Research/Research - AI Chatbot TA/github/data-generator/QA_generation_from_textbook/reinforcement_learning/GPT-3_questions_cleaned.json")
questions_manual = json.load(file1)

In [6]:
sections_list = []
for p, item in enumerate(sections_data[17:]):
    subtext = item['positive_ctxs']['text']
    sections_list.append(subtext)

In [9]:
questions = questions_manual

In [ ]:
flattened_list = [element for sublist in questions for element in sublist]
print(flattened_list)


In [41]:
questions_fin = flattened_list

In [51]:
questions_fin[19]

' What is the input/output interface of the game controller?'

In [ ]:
answer_list = []
for q, ques in enumerate(questions_fin[14:20]):
    print(q)
    answers = generate_answers(ques)
    answer_list.append(answers)

In [ ]:
qa_data = []
for q, ques in enumerate(questions_fin[14:20]):
    data = {}
    data['question'] = ques
    print(ques)
    print(answer_list[q])
    data['answers'] = answer_list[q]
    qa_data.append(data)


In [63]:
qa_data

[{'question': ' What is a 4-input priority encoder?',
  'answers': ['\n\nA 4-input priority encoder is a digital circuit used to convert a set of four binary inputs into an encoded output. It assigns priority to each input, so that only one of the inputs can be active at any given time. For example, if three of the four inputs are active, the highest priority input will be decoded as the output.',
   '\n\nA 4-input priority encoder is a digital circuit that converts a set of binary inputs into an encoded output. It works by assigning each input a priority level, and then outputs the code corresponding to the highest priority input. For example, if inputs A, B, C and D have respective priorities 1, 2, 3 and 4 then if A is active (high) it will output code 1; if B is active it will output code 2; if C is active it will output code 3; and if D is active it will output code 4.',
   '\n\nAnswer: A 4-input priority encoder is a digital logic circuit that takes four binary inputs and produces